In [1]:
import os
os.getcwd()

'/workspace'

In [ ]:
import os
from nemo_curator.download import download_wikipedia
from dask.distributed import Client, LocalCluster


cur_dir = os.getcwd()
print(cur_dir)
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"

cluster = LocalCluster(n_workers=48, processes=True, memory_limit='38GB')
client = Client(cluster)

# Output
download_base_directory= os.path.join(data_dir, "wiki_downloads")
download_output_directory = os.path.join(download_base_directory, "data")
print(download_output_directory)

# Relevant parameters
dump_date = "20241001"
language = 'ja'
url_limit = 1  # 1 file　(jawiki-20240801-pages-articles-multistream1.xml-p1p114794.bz2)

res = download_wikipedia(
    download_output_directory,
    language=language, 
    dump_date=dump_date,
    url_limit=url_limit
).df.compute()

#client.cluster.close()
#client.shutdown()

In [1]:
import os
import time
from dask.distributed import Client, LocalCluster

from nemo_curator import ScoreFilter,Modify
from nemo_curator.datasets import DocumentDataset
from nemo_curator.filters import FastTextLangId
from nemo_curator.modifiers import UnicodeReformatter
from nemo_curator.utils.file_utils import separate_by_metadata


cur_dir = os.getcwd()
print(cur_dir)
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"

# 前の処理でclusterを落としている場合は以下をアンコメントして再度起動してください
cluster = LocalCluster(n_workers=48, processes=True, memory_limit='38GB')
client = Client(cluster)

# Input path
# multilingual_data_path = "./wiki_downloads/data/jawiki-20240801-pages-articles-multistream1.xml-p1p114794.bz2.jsonl"
multilingual_data_path = "wiki_downloads/data/jawiki-20241001-pages-articles-multistream1.xml-p1p114794.bz2.jsonl"
# multilingual_data_path = "wiki_downloads/data/jawiki-20241001-pages-articles-multistream1.added_meta.xml-p1p114794.bz2.jsonl"
multilingual_data_full_path = os.path.join(data_dir, multilingual_data_path)

# Output path
language_base_output_path = os.path.join(data_dir,"language_sep")
language_data_output_path = os.path.join(language_base_output_path,"data")
language_separated_output_path = os.path.join(language_data_output_path,"language")

# Fasttext model path
model_path = language_base_output_path

# Define key in output .jsonl files to store the language information
language_field = "language"
# print(f"{model_path=}")

!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -P {model_path}
t0 = time.time()

# Load dataset 
# multilingual_dataset = DocumentDataset.read_json(multilingual_data_full_path, add_filename=True, dtype={"text": str, "title": str, "id": "int64", "url":str, "language":str, "source_id": str, "filename": str})
multilingual_dataset = DocumentDataset.read_json(multilingual_data_full_path, add_filename=True, input_meta={**{"text": str}, **{"title": str}, **{"id": "int64"}, **{"url":str}, **{"language":str}, **{"source_id": str}, **{"filename": str}})
# multilingual_dataset = DocumentDataset.read_json(multilingual_data_full_path, add_filename=True, input_meta={"id": int})
# # multilingual_dataset = DocumentDataset.read_json(multilingual_data_full_path, add_filename=True)

# Define Language separation pipeline
lang_filter = FastTextLangId(os.path.join(model_path,'lid.176.bin'))
language_id_pipeline = ScoreFilter(lang_filter, score_field=language_field, score_type='object')
filtered_dataset = language_id_pipeline(multilingual_dataset)
# print(f"{filtered_dataset=}")
# print(f"{filtered_dataset.df=}")
# print(f"{filtered_dataset.df[language_field]=}")

# The language separation pipeline will produce a result looks like ['JA',0.96873], we only want to keep the 'JA' label and drop the detailed classifier score
filtered_dataset.df[language_field] = filtered_dataset.df[language_field].apply(lambda score: score[1],meta = (language_field, 'object'))
# print(f"{filtered_dataset.df[language_field]=}")
# Split the dataset to corresponding language sub-folders
language_stats = separate_by_metadata(filtered_dataset.df, language_separated_output_path, metadata_field=language_field).compute()

print(f"Time taken for splitting language:{time.time()-t0}")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/


2024-11-23 13:26:19,905 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,910 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,915 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,921 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,926 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,935 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,942 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 38GB due to system memory limit of 31.29 GiB
2024-11-23 13:26:19,949 - d

--2024-11-23 13:26:32--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.166.244.93, 3.166.244.8, 3.166.244.49, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.166.244.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘/workspace/language_sep/lid.176.bin.4’

lid.176.bin.4       100%[===================>] 125.18M  11.5MB/s    in 12s     

2024-11-23 13:26:44 (10.7 MB/s) - ‘/workspace/language_sep/lid.176.bin.4’ saved [131266198/131266198]

Reading 1 files
read_kwargs={'lines': True, 'dtype': False}


df[metadata_field]=0        JA
1        JA
2        JA
3        JA
4        JA
         ..
59649    JA
59650    JA
59651    JA
59652    JA
59653    JA
Name: language, Length: 59628, dtype: object
Time taken for splitting language:126.00551414489746


In [2]:
t0 = time.time()

# Define desired language
target_language = "JA"

# Output path
lang_sep_cleaned_data_output_path = os.path.join(language_data_output_path, "cleaned")

# Read the language specific data and fix the unicode in it
lang_data_path = os.path.join(language_separated_output_path, target_language)
lang_data = DocumentDataset.read_json(lang_data_path,add_filename=True)

cleaner = Modify(UnicodeReformatter())
cleaned_data = cleaner(lang_data)

# Write the cleaned_data
cleaned_data.to_json(lang_sep_cleaned_data_output_path, write_to_filename=True)

print(f"Time taken for fixing unicode:{time.time()-t0}")

Reading 1 files
read_kwargs={'lines': True, 'dtype': False}
read_kwargs={'lines': True, 'dtype': False}
Writing to disk complete for 1 partitions
Time taken for fixing unicode:461.9042181968689


In [5]:
import os
import time

from nemo_curator import AddId
from nemo_curator.datasets import DocumentDataset


cur_dir = os.getcwd()
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"

# Input
add_id_input_data_dir = os.path.join(data_dir, "language_sep/data/cleaned")

# Output
added_id_output_path = os.path.join(data_dir, "add_id/cleaned")

# Format of output ID will be <prefix>_<id>, Define prefix here
add_ID_id_prefix="JA_wiki"

t0 = time.time()
# Read input files
dataset = DocumentDataset.read_json(add_id_input_data_dir,add_filename=True)

# Run AddID() on the input dataset
add_id = AddId(id_field='id',id_prefix=add_ID_id_prefix,start_index=0)
id_dataset = add_id(dataset)

# Output files
id_dataset.to_json(added_id_output_path, write_to_filename=True)

print(f"Time taken for add ID:{time.time()-t0}")

Reading 1 files
read_kwargs={'lines': True, 'dtype': False}
Writing to disk complete for 1 partitions
Time taken for add ID:48.32787084579468


In [6]:
%env DASK_DATAFRAME__QUERY_PLANNING=False

import os
import time
import pandas as pd

from nemo_curator.datasets import DocumentDataset
from nemo_curator.modules import ExactDuplicates
from nemo_curator.utils.distributed_utils import get_client, get_num_workers

def pre_imports():
    import cudf 


cur_dir = os.getcwd()
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"


client = get_client(cluster_type='gpu', set_torch_to_use_rmm=False)
print(f"Number of dask worker:{get_num_workers(client)}")
client.run(pre_imports)

# Input
exact_dedup_input_dataset_dir = os.path.join(data_dir, "add_id/cleaned")

# Output
exact_dedup_base_output_path = os.path.join(data_dir, "exact_dedup")
exact_dedup_log_dir = os.path.join(exact_dedup_base_output_path, 'log')
exact_dedup_output_dir = os.path.join(exact_dedup_base_output_path, 'data')

# Parameters for ExactDuplicates()
exact_dedup_dataset_id_field = "id"
exact_dedup_dataset_text_field = "text"

!mkdir -p {exact_dedup_log_dir}
!mkdir -p {exact_dedup_output_dir}

t0 = time.time()
# Read input dataset
input_dataset = DocumentDataset.read_json(exact_dedup_input_dataset_dir, backend='cudf')

# Run exact deduplication to the input
exact_dup = ExactDuplicates(
    logger=exact_dedup_log_dir,
    id_field=exact_dedup_dataset_id_field,
    text_field=exact_dedup_dataset_text_field,
    hash_method="md5",
    cache_dir=exact_dedup_output_dir  # Duplicated document ID list is output to the cache_dir
)
duplicates = exact_dup(dataset=input_dataset)

print(f"Number of exact duplicated file:{len(duplicates)}")
print(f"Time taken for exact duplicate:{time.time()-t0}")

exact_dedup_res = pd.read_parquet(os.path.join(exact_dedup_output_dir, "_exact_duplicates.parquet"))
print(f"Number of exact duplicated document:{len(exact_dedup_res)}")
exact_dedup_res.head()

exact_dedup_res.groupby('_hashes')['id'].agg(lambda x: ' '.join(x)).reset_index().head()

env: DASK_DATAFRAME__QUERY_PLANNING=False


/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35027 instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(


Number of dask worker:1
Reading 1 files
Number of exact duplicated file:2
Time taken for exact duplicate:15.952196598052979
Number of exact duplicated document:2


,_hashes,id
0,0d3ffcf18f61d821db277db357fa7b0e,JA_wiki-0000052756 JA_wiki-0000052758


In [ ]:
%env DASK_DATAFRAME__QUERY_PLANNING=False

import os
import time
import pandas as pd

from nemo_curator import FuzzyDuplicates, FuzzyDuplicatesConfig
from nemo_curator.datasets import DocumentDataset
from nemo_curator.utils.distributed_utils import get_client, get_num_workers

import dask

def pre_imports():
    import cudf 

    
cur_dir = os.getcwd()
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"

# client = get_client(cluster_type='gpu', set_torch_to_use_rmm=False)
print(f"Number of dask worker:{get_num_workers(client)}")
# client.run(pre_imports)

# Input
fuzzy_dedup_data_path = os.path.join(data_dir, "add_id/cleaned")

# Output
fuzzy_dedup_base_output_path = os.path.join(data_dir, "fuzzy_wrapper")
fuzzy_dedup_log_dir = os.path.join(fuzzy_dedup_base_output_path, 'log')
fuzzy_dedup_cache_dir = os.path.join(fuzzy_dedup_base_output_path, 'cache')
fuzzy_dedup_output_dir = os.path.join(fuzzy_dedup_base_output_path, 'data')

# Relevant parameters
id_field = 'id'
text_field = 'text'
filetype = "parquet"

!mkdir -p {fuzzy_dedup_base_output_path}
!mkdir -p {fuzzy_dedup_log_dir}
!mkdir -p {fuzzy_dedup_cache_dir}
!mkdir -p {fuzzy_dedup_output_dir}

#!rm -r {fuzzy_dedup_cache_dir}

with dask.config.set({"dataframe.backend": 'cudf'}):
        
    t0 = time.time()
        
    input_dataset = DocumentDataset.read_json(fuzzy_dedup_data_path, backend='cudf')
    fuzzy_dedup_config = FuzzyDuplicatesConfig(
        cache_dir=fuzzy_dedup_cache_dir,
        id_field=id_field,
        text_field=text_field,
        seed=42,  # Use the seed set in Minhash section for consistency
        char_ngrams=5,
        num_buckets=20,
        hashes_per_bucket=13,
        use_64_bit_hash=False,
        buckets_per_shuffle=5,
        false_positive_check=True,
        num_anchors=2,
        jaccard_threshold=0.8,
    )
    fuzzy_dup = FuzzyDuplicates(logger=fuzzy_dedup_log_dir, config=fuzzy_dedup_config)
    duplicates = fuzzy_dup(dataset=input_dataset)
        
    duplicates.to_parquet(fuzzy_dedup_output_dir, write_to_filename=False)
       
    print(f"Time taken for Connected Component: {time.time()-t0} s")    
        
fuzzy_dedup_res = pd.read_parquet(fuzzy_dedup_output_dir)
fuzzy_dedup_res.head()

env: DASK_DATAFRAME__QUERY_PLANNING=False


/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41351 instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(


Number of dask worker:1
Reading 1 files


/opt/NeMo-Curator/nemo_curator/modules/config.py:91: UserWarning: Identifying false positives during the Minhash deduplication is computationally expensive. For improved performance consider setting this to False
  warnings.warn(


Stage1: Starting Minhash + LSH computation
Stage1: Minhash + LSH complete!
Stage2 (False Positive Check): Starting Map_Buckets
Stage2 (False Postive Check): Map_Buckets Complete!
Stage3 (False Postive Check): Shuffle docs


...

Stage3 (False Postive Check): Shuffle docs complete!
Stage4 (False Postive Check): Jaccard Similarity in Buckets


Stage4 (False Postive Check): Jaccard Similarity in Buckets Complete!
Stage5: Connected Components across buckets
batch_id = 0/1, time = 0.4316842555999756
[1732369704.916822] [docker-desktop:19625:0]          parser.c:2305 UCX  WARN  unused environment variable: UCX_MEMTYPE_CACHE (maybe: UCX_MEMTYPE_CACHE?)
[1732369704.916822] [docker-desktop:19625:0]          parser.c:2305 UCX  WARN  (set UCX_WARN_UNUSED_ENV_VARS=n to suppress this warning)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1324: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cudf/core/reshape.py:350: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  warnings.warn(


# of groups 95
# of docs removed 15
assert num_nodes:110==labels_df:110 passed
Stage5: Connected Components across buckets complete!
Writing to disk complete for 1 partitions
Time taken for Connected Component: 57.66777467727661 s


,id,group
0,JA_wiki-0000025154,24
1,JA_wiki-0000045605,25
2,JA_wiki-0000026301,26
3,JA_wiki-0000026251,27
4,JA_wiki-0000025368,28


In [8]:
import os
import pandas as pd

from nemo_curator.datasets import DocumentDataset


cur_dir = os.getcwd()
# data_dir = f"{cur_dir}/"
data_dir = "/workspace/"

# Input
dataset_dir = os.path.join(data_dir, "add_id/cleaned")
exact_dedup_output_dir= os.path.join(data_dir, "exact_dedup/data")
fuzzy_dedup_output_dir= os.path.join(data_dir, "fuzzy_wrapper/data")

# Output
dudped_output_dir = os.path.join(data_dir, "remove_duplicate/result.parquet")

# Relevant parameters
input_id_field = 'id'
id_prefix = "JA_wiki"

!mkdir -p {dudped_output_dir}

#Load .jsonl dataset (GPUメモリが足りない場合はbackend='pandas'へ変更してください)
input_dataset = DocumentDataset.read_json(dataset_dir, backend='cudf')

# Load exact deduplicate result and extract list of duplicated document ID　(GPUメモリが足りない場合はbackend='pandas'へ変更してください)
exact_duplicates = DocumentDataset.read_parquet(os.path.join(exact_dedup_output_dir, "_exact_duplicates.parquet"), backend='cudf')
exact_docs_to_remove = exact_duplicates.df.map_partitions(
    lambda x: x[x._hashes.duplicated(keep="first")]
)

# Remove the duplicated document from input dataset
result = input_dataset.df[
    ~input_dataset.df[input_id_field].isin(exact_docs_to_remove[input_id_field].compute())
]

# Loads result from fuzzy dedup wrapper
fuzzy_duplicates = pd.read_parquet(fuzzy_dedup_output_dir)

# Generate list of near duplicate document ID
fuzzy_docs_to_remove = fuzzy_duplicates.drop_duplicates(subset=['group'], keep='first')

# Remove near duplicates
result = result[~result[input_id_field].isin(fuzzy_docs_to_remove[input_id_field])]

# Save final result to local (backend='pandas'の場合は、write_to_filename=Trueをコメントアウトしてください)
result.to_parquet(dudped_output_dir, write_to_filename=True)

res = pd.read_parquet(dudped_output_dir)
print(f"Length of duplicate removed dataset:{len(res)}")

Reading 1 files
Reading 1 files
Length of duplicate removed dataset:59508


In [3]:
import asyncio
import nest_asyncio
import os
from dotenv import load_dotenv

from openai import AsyncOpenAI

cur_dir = os.getcwd()
data_dir = f"{cur_dir}/"
dotenv_path = os.path.join(data_dir, ".env")
load_dotenv(dotenv_path, verbose=True)
nvidia_api_key = os.getenv("NVIDIA_API_KEY")

client = AsyncOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=nvidia_api_key  # 取得したAPIキーを入力してください
)

n_subtopics = 2
n_questions = 2
topic = "機械学習"

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
トピックが与えられた場合、そのトピックに関連する {n_subtopics} のサブトピックのリストを生成してください。
トピックは：{topic}
リストは番号なしで、サブトピックの説明なしでなければなりません。サブトピックはコンマで区切られる必要があります。リスト以外のテキストは存在してはなりません。
"""

QUESTION_PROMPT_TEMPLATE = """\
トピックが与えられた場合、そのトピックに関して{n_questions}個の質問を生成してください。
トピックは：{sub_topic}
リスト形式で、質問は改行文字で区切られる必要があります。リスト以外のテキストは存在してはなりません。
"""

RESPONSE_PROMPT_TEMPLATE = """\
質問が与えられた場合、その質問に対して考えられる2つの回答を生成してください。
質問は：{question}
リスト形式は以下の形式である必要があります：

RESPONSE A: ここに回答Aのテキストを入力
RESPONSE B: ここに回答Bのテキストを入力
"""

In [4]:
# data_dir = "/workspace/"

# print(os.environ["NVIDIA_API_KEY"])
# print(os.environ.get("NVIDIA_API_KEY"))

# generate sub topics
async def generate_subtopics(client, topic, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(topic=topic, n_subtopics=n_subtopics)
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role" : "user",
             "content" : prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response

subtopics = await generate_subtopics(client, topic, n_subtopics)
subtopic_list = subtopics.choices[0].message.content.split(",")
print(subtopic_list)

['ディープラーニング、強化学習']


In [5]:
# generate questions of sub topics
async def generate_questions(client, sub_topic, n_questions):
    prompt = QUESTION_PROMPT_TEMPLATE.format(sub_topic=sub_topic, n_questions=n_questions)
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role" : "user",
             "content" : prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    if hasattr(response, 'choices') and response.choices:
        return response.choices[0].message.content
    else:
        print(f"Unexpected response structure: {response}")
        return None

async def question_generator(client, subtopic_list, n_question):
    tasks = [generate_questions(client, subtopic, n_question) for subtopic in subtopic_list]
    question_list = await asyncio.gather(*tasks)
    return question_list

nest_asyncio.apply()
question_list = asyncio.run(question_generator(client, subtopic_list, n_questions))
print(question_list)

# format questions
question_list_formatted = []
for question_set in question_list:
    question_list_formatted += question_set.split("\n\n")

['ディープラーニングと強化学習の違いは何ですか？\n\nディープラーニングと強化学習を組み合わせた研究事例を教えてください。']


In [6]:
# generate response of each question
async def generate_responses(client, question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question)
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role" : "user",
             "content" : prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    if hasattr(response, 'choices') and response.choices:
        return response.choices[0].message.content
    else:
        print(f"Unexpected response structure: {response}")
        return None

async def response_generator(client, question_list):
    tasks = [generate_responses(client, question) for question in question_list]
    response_list = await asyncio.gather(*tasks)
    return response_list

question_response_list = asyncio.run(response_generator(client, question_list_formatted))
print(question_response_list)

['RESPONSE A: ディープラーニングは、主に大規模なデータセットを使用してパターンを学習し、画像や音声などのデータを認識することに重点を置いた機械学習の一種です。一方、強化学習は、エージェントが環境とやり取りし、報酬を最大化する行動を学習するプロセスに重点を置いています。\n\nRESPONSE B: ディープラーニングは、ニューラルネットワークを使用してデータの複雑なパターンを学習する手法であり、画像認識や自然言語処理などの分野で広く使用されています。強化学習は、エージェントが環境から得られる報酬に基づいて行動を学習し、ロボット工学やゲームなどの分野で応用されています。', 'RESPONSE A: AlphaGoはディープラーニングと強化学習を組み合わせた研究事例として有名です。AlphaGoは、Google DeepMindによって開発されたコンピュータープログラムで、囲碁のゲームをプレイするために設計されました。AlphaGoは、ディープラーニングを使用してゲームの状態を評価し、強化学習を使用してゲームの戦略を学習しました。AlphaGoは、2016年に世界のトッププレイヤーである李世乭を破り、コンピュータープログラムが人間の世界チャンピオンを破った初めての例となりました。\n\nRESPONSE B: 自動運転車の開発におけるディープラーニングと強化学習の組み合わせも注目されています。自動運転車は、ディープラーニングを使用して道路の状況を認識し、強化学習を使用して運転の戦略を学習します。強化学習アルゴリズムは、自動運転車がさまざまな状況で安全かつ効率的に運転する方法を学習するために使用されます。例えば、自動運転車は強化学習を使用して、道路の状況に応じて速度を調整したり、障害物を回避したりする方法を学習できます。このような研究事例は、自動運転車の安全性と性能を向上させるために重要です。']


In [7]:
# prepare question:response pair set list
question_response_pair_list = []
for question, response_set in zip(question_list_formatted, question_response_list):
    question_response_pair_list.append(
        {
            "question" : question, 
            "responses" : {
                "response_a" : {"response" : response_set.split("RESPONSE B:")[0].replace("RESPONSE A:", "").strip().split("\n\n")[-1].strip()},
                "response_b" : {"response" : response_set.split("RESPONSE B:")[-1].split("\n\n")[0].strip()}
            },
        }
    )

import json

# export to jsonl file
with open('synthetic_data.jsonl', 'w') as f:
    for item in question_response_pair_list:
        print(f"{item=}")
        f.write(json.dumps(item, ensure_ascii=False))
        f.write('\n')

item={'question': 'ディープラーニングと強化学習の違いは何ですか？', 'responses': {'response_a': {'response': 'ディープラーニングは、主に大規模なデータセットを使用してパターンを学習し、画像や音声などのデータを認識することに重点を置いた機械学習の一種です。一方、強化学習は、エージェントが環境とやり取りし、報酬を最大化する行動を学習するプロセスに重点を置いています。'}, 'response_b': {'response': 'ディープラーニングは、ニューラルネットワークを使用してデータの複雑なパターンを学習する手法であり、画像認識や自然言語処理などの分野で広く使用されています。強化学習は、エージェントが環境から得られる報酬に基づいて行動を学習し、ロボット工学やゲームなどの分野で応用されています。'}}}
item={'question': 'ディープラーニングと強化学習を組み合わせた研究事例を教えてください。', 'responses': {'response_a': {'response': 'AlphaGoはディープラーニングと強化学習を組み合わせた研究事例として有名です。AlphaGoは、Google DeepMindによって開発されたコンピュータープログラムで、囲碁のゲームをプレイするために設計されました。AlphaGoは、ディープラーニングを使用してゲームの状態を評価し、強化学習を使用してゲームの戦略を学習しました。AlphaGoは、2016年に世界のトッププレイヤーである李世乭を破り、コンピュータープログラムが人間の世界チャンピオンを破った初めての例となりました。'}, 'response_b': {'response': '自動運転車の開発におけるディープラーニングと強化学習の組み合わせも注目されています。自動運転車は、ディープラーニングを使用して道路の状況を認識し、強化学習を使用して運転の戦略を学習します。強化学習アルゴリズムは、自動運転車がさまざまな状況で安全かつ効率的に運転する方法を学習するために使用されます。例えば、自動運転車は強化学習を使用して、道路の状況に応じて速度を調整したり、障害物を回避したりする方法を学習できます。このような研究事例は、自動運転車の安全性と性能を向上させるた

In [9]:
def get_scores_from_response(openai_response_template):
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict

async def get_response_and_scores(client, question, response_content):
    messages = [
        {"role": "user","content": question},
        {"role": "assistant","content": response_content},]
    response = await client.chat.completions.create(
        model="nvidia/nemotron-4-340b-reward",
        messages=messages,
    )
    scores = get_scores_from_response(response)
    return scores

# scoring for question:response pair set
async def process_question_response_pairs(client,question_response_score_list):
    tasks = []
    for question_response_pair in question_response_score_list:
        question = question_response_pair["question"]
        
        task_a = get_response_and_scores(client, question, question_response_pair["responses"]["response_a"]["response"])
        task_b = get_response_and_scores(client, question, question_response_pair["responses"]["response_b"]["response"])
        
        tasks.append((task_a, question_response_pair, "response_a"))
        tasks.append((task_b, question_response_pair, "response_b"))
    results = await asyncio.gather(*[task[0] for task in tasks])
    
    for i, (result, task_info) in enumerate(zip(results, tasks)):
        _, question_response_pair, response_key = task_info
        question_response_pair["responses"][response_key].update(result)
question_response_score_list = question_response_pair_list.copy()
await process_question_response_pairs(client, question_response_score_list)
print(question_response_score_list)

[{'question': 'ディープラーニングと強化学習の違いは何ですか？', 'responses': {'response_a': {'response': 'ディープラーニングは、主に大規模なデータセットを使用してパターンを学習し、画像や音声などのデータを認識することに重点を置いた機械学習の一種です。一方、強化学習は、エージェントが環境とやり取りし、報酬を最大化する行動を学習するプロセスに重点を置いています。', 'helpfulness': 3.140625, 'correctness': 3.140625, 'coherence': 3.671875, 'complexity': 1.734375, 'verbosity': 1.3984375}, 'response_b': {'response': 'ディープラーニングは、ニューラルネットワークを使用してデータの複雑なパターンを学習する手法であり、画像認識や自然言語処理などの分野で広く使用されています。強化学習は、エージェントが環境から得られる報酬に基づいて行動を学習し、ロボット工学やゲームなどの分野で応用されています。', 'helpfulness': 2.9375, 'correctness': 2.859375, 'coherence': 3.625, 'complexity': 1.5703125, 'verbosity': 1.3203125}}}, {'question': 'ディープラーニングと強化学習を組み合わせた研究事例を教えてください。', 'responses': {'response_a': {'response': 'AlphaGoはディープラーニングと強化学習を組み合わせた研究事例として有名です。AlphaGoは、Google DeepMindによって開発されたコンピュータープログラムで、囲碁のゲームをプレイするために設計されました。AlphaGoは、ディープラーニングを使用してゲームの状態を評価し、強化学習を使用してゲームの戦略を学習しました。AlphaGoは、2016年に世界のトッププレイヤーである李世乭を破り、コンピュータープログラムが人間の世界チャンピオンを破った初めての例となりました。', 'helpfulness': 3.203125, 'correctness

In [10]:
# prepare question:response pair set list
question_response_pair_list = []
for question, response_set in zip(question_list_formatted, question_response_list):
    question_response_pair_list.append(
        {
            "question" : question, 
            "responses" : {
                "response_a" : {"response" : response_set.split("RESPONSE B:")[0].replace("RESPONSE A:", "").strip().split("\n\n")[-1].strip()},
                "response_b" : {"response" : response_set.split("RESPONSE B:")[-1].split("\n\n")[0].strip()}
            },
        }
    )

import json

# export to jsonl file
with open('synthetic_data.jsonl', 'w') as f:
    for item in question_response_pair_list:
        f.write(json.dumps(item, ensure_ascii=False))
        f.write('\n')

In [12]:
# running reward scoring model to evaluate the responses 
def get_scores_from_response(openai_response_template):
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict

async def get_response_and_scores(client, question, response_content):
    messages = [
        {"role": "user","content": question},
        {"role": "assistant","content": response_content},]
    response = await client.chat.completions.create(
        model="nvidia/nemotron-4-340b-reward",
        messages=messages,
    )
    scores = get_scores_from_response(response)
    return scores

# scoring for question:response pair set
async def process_question_response_pairs(client,question_response_score_list):
    tasks = []
    for question_response_pair in question_response_score_list:
        question = question_response_pair["question"]
        
        task_a = get_response_and_scores(client, question, question_response_pair["responses"]["response_a"]["response"])
        task_b = get_response_and_scores(client, question, question_response_pair["responses"]["response_b"]["response"])
        
        tasks.append((task_a, question_response_pair, "response_a"))
        tasks.append((task_b, question_response_pair, "response_b"))
    results = await asyncio.gather(*[task[0] for task in tasks])
    
    for i, (result, task_info) in enumerate(zip(results, tasks)):
        _, question_response_pair, response_key = task_info
        question_response_pair["responses"][response_key].update(result)
question_response_score_list = question_response_pair_list.copy()
await process_question_response_pairs(client, question_response_score_list)
print(question_response_score_list)

[{'question': 'ディープラーニングと強化学習の違いは何ですか？', 'responses': {'response_a': {'response': 'ディープラーニングは、主に大規模なデータセットを使用してパターンを学習し、画像や音声などのデータを認識することに重点を置いた機械学習の一種です。一方、強化学習は、エージェントが環境とやり取りし、報酬を最大化する行動を学習するプロセスに重点を置いています。', 'helpfulness': 3.140625, 'correctness': 3.140625, 'coherence': 3.671875, 'complexity': 1.734375, 'verbosity': 1.3984375}, 'response_b': {'response': 'ディープラーニングは、ニューラルネットワークを使用してデータの複雑なパターンを学習する手法であり、画像認識や自然言語処理などの分野で広く使用されています。強化学習は、エージェントが環境から得られる報酬に基づいて行動を学習し、ロボット工学やゲームなどの分野で応用されています。', 'helpfulness': 2.9375, 'correctness': 2.859375, 'coherence': 3.625, 'complexity': 1.5703125, 'verbosity': 1.3203125}}}, {'question': 'ディープラーニングと強化学習を組み合わせた研究事例を教えてください。', 'responses': {'response_a': {'response': 'AlphaGoはディープラーニングと強化学習を組み合わせた研究事例として有名です。AlphaGoは、Google DeepMindによって開発されたコンピュータープログラムで、囲碁のゲームをプレイするために設計されました。AlphaGoは、ディープラーニングを使用してゲームの状態を評価し、強化学習を使用してゲームの戦略を学習しました。AlphaGoは、2016年に世界のトッププレイヤーである李世乭を破り、コンピュータープログラムが人間の世界チャンピオンを破った初めての例となりました。', 'helpfulness': 3.203125, 'correctness